# Classification of Emission Line Galaxies with SDSS & WISE

## Imports

In [142]:
import pandas as pd
import numpy as np
from collections import Counter

%matplotlib
import matplotlib.pyplot as plt
plt.style.use('bmh')

import seaborn as sns
sns.set_theme()
sns.set_context('paper')

import calc_kcor

Using matplotlib backend: MacOSX


In [4]:
galaxies = pd.read_csv('all_galaxies.csv', header=1)
galaxies

,specobjid,plate,mjd,fiberID,subclass,z,oiii_5007_flux,oiii_5007_flux_err,h_alpha_flux,h_alpha_flux_err,...,neiii_3869_flux,neiii_3869_flux_err,oii_3726_flux,oii_3726_flux_err,oii_3729_flux,oii_3729_flux_err,psfMag_g,psfMagErr_g,psfMag_z,psfMagErr_z
0,305174431683078144,271,51883,202,STARBURST,0.063123,92.12244,2.618963,183.45430,2.865695,...,3.729844,3.002521,87.23842,4.977502,105.56950,5.273711,20.34747,0.036948,19.70755,0.085191
1,305144744869128192,271,51883,94,STARBURST,0.049476,76.97161,2.971411,1002.40600,8.275866,...,12.437810,4.509258,178.42940,8.317794,194.33960,8.494857,18.67184,0.026452,17.71873,0.018955
2,305131275851687936,271,51883,45,STARBURST,0.094921,56.61370,3.481388,669.67180,6.949439,...,10.556750,4.273049,89.65728,7.260168,97.16902,6.971335,18.89609,0.031938,17.54852,0.022769
3,306322322795554816,272,51941,282,STARBURST,0.043465,60.38306,3.433703,1448.79300,11.199520,...,9.628464,5.808692,117.33540,10.122940,133.95710,10.596660,18.52655,0.020641,17.54534,0.019796
4,305135124142385152,271,51883,59,STARFORMING,0.060961,51.07178,2.564878,143.97120,2.619117,...,4.465749,3.506365,65.12830,5.723801,63.45277,5.713418,20.21657,0.026394,19.33278,0.058647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,814075147597146112,723,52201,180,NaN,0.082321,11.12208,2.495317,64.71101,3.172045,...,6.175425,2.983800,35.02275,4.922418,16.16962,4.913709,19.38489,0.085007,17.55874,0.065260
29996,814072948573890560,723,52201,172,STARFORMING,0.082470,10.98368,1.989818,113.41780,2.669237,...,2.705884,2.598446,28.19714,3.695714,13.21524,3.605189,19.27386,0.017670,17.90965,0.026622
29997,814066626382030848,723,52201,149,STARBURST,0.122621,29.49972,2.428270,666.63770,8.007282,...,9.991094,2.794462,92.43250,4.480760,76.38293,4.320314,19.26258,0.017628,18.24440,0.028842
29998,814057280533194752,723,52201,115,STARBURST,0.064417,78.22813,2.803072,627.94100,6.362722,...,9.178064,3.713648,157.44550,7.112261,216.14780,7.913051,18.90501,0.016792,18.30284,0.031095


# Task 1

## Distribution of the data

In [5]:
Counter(galaxies['subclass'])

Counter({'STARBURST': 6348,
         'STARFORMING': 14851,
         nan: 5575,
         'AGN': 1993,
         'STARFORMING BROADLINE': 112,
         'BROADLINE': 549,
         'STARBURST BROADLINE': 230,
         'AGN BROADLINE': 342})

In [6]:
table_cols = ['specobjid', 'subclass', 'z',
       'oiii_5007_flux', 'oiii_5007_flux_err', 
       'h_alpha_flux', 'h_alpha_flux_err', 
       'h_beta_flux', 'h_beta_flux_err', 
       'nii_6584_flux', 'nii_6584_flux_err', 
       'sii_6717_flux', 'sii_6717_flux_err', 
       'sii_6731_flux', 'sii_6731_flux_err', 
       'oi_6300_flux', 'oi_6300_flux_err', 
       'h_alpha_reqw', 'h_alpha_reqw_err',
       'neiii_3869_flux', 'neiii_3869_flux_err', 
       'oii_3726_flux', 'oii_3726_flux_err', 
       'oii_3729_flux', 'oii_3729_flux_err', 
       'psfMag_g', 'psfMagErr_g', 
       'psfMag_z', 'psfMagErr_z']

# galaxies[table_cols].to_latex('./final_project_report/tab_all_galaxies.tex')

In [106]:
cols = ['oiii_5007_flux', 'h_beta_flux', 'nii_6584_flux', 'sii_6717_flux', 
        'sii_6731_flux','oi_6300_flux', 'h_alpha_flux', 'neiii_3869_flux', 
        'oii_3726_flux', 'oii_3729_flux']

for col in cols:
    sns.displot(data=galaxies, x=col, log_scale=True, kde=True)

    plt.xlabel(col + r' [$10^{-17}$ erg s$^{-1}$ cm$^{-2}$]')

    # plt.savefig('./data_histograms/distr_' + col + '.pdf')

    plt.show()

/Users/nicolasgalvarinoguerravaras/anaconda3/lib/python3.7/site-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


In [108]:
cols = ['psfMag_g', 'psfMag_z']

for col in cols:
    sns.displot(data=galaxies, x=col, log_scale=False, kde=True)

    plt.xlabel(col + r' [mag]')

    # plt.savefig('./data_histograms/distr_' + col + '.pdf')

    plt.show()

In [8]:
sns.displot(data=galaxies, x='h_alpha_reqw', log_scale=False, kde=True)

plt.xlabel(r'h_alpha_reqw [Ang]')

plt.xscale('symlog')

# plt.savefig('./data_histograms/distr_h_alpha_reqw.pdf')

plt.show()

In [9]:
sns.displot(data=galaxies, x='z', log_scale=False, kde=True)

plt.xlabel(col)
# plt.savefig('distr_redshift.pdf')

plt.show()

## BPT Diagram

In [10]:
galaxies.columns

Index(['specobjid', 'plate', 'mjd', 'fiberID', 'subclass', 'z',
       'oiii_5007_flux', 'oiii_5007_flux_err', 'h_alpha_flux',
       'h_alpha_flux_err', 'h_beta_flux', 'h_beta_flux_err', 'nii_6584_flux',
       'nii_6584_flux_err', 'sii_6717_flux', 'sii_6717_flux_err',
       'sii_6731_flux', 'sii_6731_flux_err', 'oi_6300_flux',
       'oi_6300_flux_err', 'h_alpha_reqw', 'h_alpha_reqw_err',
       'neiii_3869_flux', 'neiii_3869_flux_err', 'oii_3726_flux',
       'oii_3726_flux_err', 'oii_3729_flux', 'oii_3729_flux_err', 'psfMag_g',
       'psfMagErr_g', 'psfMag_z', 'psfMagErr_z'],
      dtype='object')

In [11]:
# [NII] / H alpha
galaxies['nii_h_alpha'] = galaxies.apply(lambda x: x['nii_6584_flux'] / x['h_alpha_flux'], axis=1)
galaxies['nii_h_alpha_log'] = galaxies.apply(lambda x: np.log10(x['nii_h_alpha']), axis=1)

# [OIII] / H beta
galaxies['oiii_h_beta'] = galaxies.apply(lambda x: x['oiii_5007_flux'] / x['h_beta_flux'], axis=1)
galaxies['oiii_h_beta_log'] = galaxies.apply(lambda x: np.log10(x['oiii_h_beta']), axis=1)

# [SII] / H alpha
galaxies['sii6717_h_alpha'] = galaxies.apply(lambda x: x['sii_6717_flux'] / x['h_alpha_flux'], axis=1)
galaxies['sii6717_h_alpha_log'] = galaxies.apply(lambda x: np.log10(x['sii6717_h_alpha']), axis=1)

galaxies['sii6731_h_alpha'] = galaxies.apply(lambda x: x['sii_6731_flux'] / x['h_alpha_flux'], axis=1)
galaxies['sii6731_h_alpha_log'] = galaxies.apply(lambda x: np.log10(x['sii6731_h_alpha']), axis=1)

# [OIII] / [OI]
galaxies['oi_h_alpha'] = galaxies.apply(lambda x: x['oi_6300_flux'] / x['h_alpha_flux'], axis=1)
galaxies['oi_h_alpha_log'] = galaxies.apply(lambda x: np.log10(x['oi_h_alpha']), axis=1)

### BPT Diagram with SDSS Spectral Subclasses

In [12]:
plt.figure(figsize=(10, 8))

sns.scatterplot(data=galaxies, x='nii_h_alpha', y='oiii_h_beta', palette='colorblind', 
                hue='subclass', style='subclass', size='subclass', sizes=(80, 20))

plt.xlabel(r'[NII] / H$\alpha$', fontsize=13)
plt.ylabel(r'[OIII] / H$\beta$', fontsize=13)

plt.xscale('log')
plt.yscale('log')

plt.legend(loc='lower left', fontsize=10)

# plt.savefig('./BPT_Diagrams/BPT_diagram_sdss_subclasses.pdf')

plt.show()

### [NII]/H $\alpha$ vs. [OIII]/H $\beta$

#### Kauffman et al. 2003

In [13]:
def BPT_class_Kauff(oiii_h_beta, nii_h_alpha):
    # second condition is to avoid double lines
    if (np.log10(oiii_h_beta) < 0.61 / (np.log10(nii_h_alpha) - 0.05) + 1.3) and (np.log10(nii_h_alpha) < 0.0):
        return 'Star Forming'
    else:
        return 'AGN'

#### Kewley et al. 2001

In [14]:
def BPT_class_Kew(oiii_h_beta, nii_h_alpha):
    if np.log10(oiii_h_beta) < 0.61 / (np.log10(nii_h_alpha) - 0.47) + 1.19 and np.log10(nii_h_alpha) < 0.4:
        return 'Star Forming'
    else:
        return 'AGN'


In [15]:
galaxies['BPT Kauffman'] = galaxies.apply(lambda x: BPT_class_Kauff(x['oiii_h_beta'], x['nii_h_alpha']), axis=1)
galaxies['BPT Kewley'] = galaxies.apply(lambda x: BPT_class_Kew(x['oiii_h_beta'], x['nii_h_alpha']), axis=1)

In [16]:
def BPT_final_class(row):
    if row['BPT Kauffman'] != row['BPT Kewley']:
        return 'Composite'
    else:  # they are equal
        return row['BPT Kewley']

In [17]:
galaxies['BPT final'] = galaxies.apply(lambda x: BPT_final_class(x), axis=1)
Counter(galaxies['BPT final'])

Counter({'Star Forming': 18622, 'Composite': 5665, 'AGN': 5713})

In [18]:
def plot_BPT(data, x, y, hue, xlabel, ylabel, Kauffman=True, Kewley=True, ylim=(-2.0, 2.0), save=False, save_name=None):
    
    plt.figure(figsize=(12, 8))

    plt.ylim(ylim)

    sns.scatterplot(data=data, x=x, y=y, palette='colorblind', 
                    hue=hue, style='BPT final', size=hue, sizes=(30, 15))

    # ------------------------------------------------------------------------------------------------
    # Kauffman
    if Kauffman:
        x_to_plot = np.linspace(galaxies[x].min(), 0)
        plt.plot(x_to_plot, 0.61 / (x_to_plot - 0.05) + 1.3, label='Kauffman et al. 2003', 
                linewidth=1.5, alpha=0.8, c='k', linestyle='dashdot')

    # ------------------------------------------------------------------------------------------------
    # Kewley
    if Kewley:
        x_to_plot_1 = np.linspace(galaxies[x].min(), 0.4)
        plt.plot(x_to_plot_1, 0.61 / (x_to_plot_1 - 0.47) + 1.19, label='Kewley et al. 2001', 
                linewidth=1.5, alpha=0.8, c='k', linestyle='dashed')

    # ------------------------------------------------------------------------------------------------

    plt.xlabel(xlabel, fontsize=13)
    plt.ylabel(ylabel, fontsize=13)

    plt.legend(loc='lower left', fontsize=12)

    if save:
        plt.savefig('./BPT_Diagrams/' + save_name + '.pdf')

    plt.show()

In [19]:
plot_BPT(galaxies, 'nii_h_alpha_log', 'oiii_h_beta_log', 'BPT final', 
         r'log [NII] / H$\alpha$', r'log [OIII] / H$\beta$', 
         Kauffman=True, Kewley=True)#, save=True, save_name='BPT_niiHa_oiiiHb')

### [SII]/H $\alpha$ vs. [OIII]/H $\beta$

In [20]:
plot_BPT(galaxies, 'sii6717_h_alpha_log', 'oiii_h_beta_log', 'BPT final', 
         r'log [SII] 6717 / H$\alpha$', r'log [OIII] / H$\beta$', 
         Kauffman=True, Kewley=True)#, save=True, save_name='BPT_sii6717Ha_oiiiHb')

In [21]:
plot_BPT(galaxies, 'sii6731_h_alpha_log', 'oiii_h_beta_log', 'BPT final', 
         r'log [SII] 6731 / H$\alpha$', r'log [OIII] / H$\beta$', 
         Kauffman=True, Kewley=True)#, save=True, save_name='BPT_sii6731Ha_oiiiHb')

In [22]:
galaxies['sii_h_alpha'] = galaxies.apply(lambda x: (x['sii6717_h_alpha'] + x['sii6731_h_alpha']) / 2, axis=1)
galaxies['sii_h_alpha_log'] = galaxies.apply(lambda x: np.log10(x['sii_h_alpha']), axis=1)

In [23]:
plot_BPT(galaxies, 'sii_h_alpha_log', 'oiii_h_beta_log', 'BPT final', 
         r'log [SII] 6717, 6731 / H$\alpha$', r'log [OIII] / H$\beta$', 
         Kauffman=True, Kewley=True)#, save=True, save_name='BPT_sii67176731Ha_oiiiHb')

### [OIII]/H $\beta$ vs [OI]/H $\alpha$

In [24]:
plot_BPT(galaxies, 'oi_h_alpha_log', 'oiii_h_beta_log', 'BPT final', 
         r'log [OI] / H$\alpha$', r'log [OIII] / H$\beta$', 
         Kauffman=True, Kewley=True)#, save=True, save_name='BPT_oiHa_oiiiHb')

## WHAN Diagram

In [25]:
galaxies['h_alpha_reqw_log'] = galaxies.apply(lambda x: np.log10(x['h_alpha_reqw']), axis=1)
galaxies['h_alpha_reqw_abs'] = galaxies.apply(lambda x: np.abs(x['h_alpha_reqw']), axis=1)

/Users/nicolasgalvarinoguerravaras/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log10
  """Entry point for launching an IPython kernel.


In [26]:
galaxies[['h_alpha_reqw', 'h_alpha_reqw_abs', 'h_alpha_reqw_log']].describe()

,h_alpha_reqw,h_alpha_reqw_abs,h_alpha_reqw_log
count,30000.000000,30000.000000,1203.000000
mean,-28.843906,28.879554,-0.515345
std,48.168780,48.147414,0.474520
min,-1986.580000,0.000202,-3.694147
25%,-34.486722,7.545384,-0.714283
50%,-18.274215,18.274215,-0.405763
75%,-7.545384,34.486722,-0.184946
max,1.886299,1986.580000,0.275611


In [27]:
plt.figure(figsize=(12, 8))

sns.scatterplot(data=galaxies, x='nii_h_alpha', y='h_alpha_reqw_abs', palette='colorblind', 
                hue='BPT final', style='BPT final', size='BPT final', sizes=(30, 15))

plt.ylim(10**(-4.2), 10**(3.2))
plt.xlim(10**(-3), 10)

plt.vlines(10**(-0.4), 10**(-4.2), 10**(3.2), linewidth=1.2, alpha=0.8, colors='k', linestyle='solid', 
           label=r'[NII] / H$\alpha = 0.4$')

plt.hlines(6, 10**(-0.4), 10, linewidth=1.5, alpha=0.8, colors='k', linestyle='dashdot', 
           label=r'$W_{H_{\alpha}} = 6$ [Ang], Kewley+2006')
plt.hlines(3, 10**(-0.4), 10, linewidth=1.5, alpha=0.8, colors='k', linestyle='dashed', 
           label=r'$W_{H_{\alpha}} = 3$ [Ang]')

plt.xlabel(r'log [NII] / H$\alpha$', fontsize=13)
plt.ylabel(r'log $W_{H_{\alpha}}$ [Ang]', fontsize=13)

plt.xscale('log')
plt.yscale('log')

plt.legend(loc='lower left', fontsize=10)

# plt.savefig('WHAN_diagram.pdf')

plt.show()

# Task 2

In [141]:
# [NeIII] / [OII]
galaxies['neiii_oii'] = galaxies.apply(lambda x: x['neiii_3869_flux'] / ((x['oii_3726_flux'] + x['oii_3729_flux']) / 2), axis=1)
galaxies['neiii_oii_log'] = galaxies.apply(lambda x: np.log10(x['neiii_oii']), axis=1)

## Redshift k-Correction

In [144]:
galaxies['g_z_color'] = galaxies['psfMag_g'] - galaxies['psfMag_z']

In [145]:
galaxies['g_z_color_zkcorr'] = galaxies.apply(lambda x: calc_kcor.calc_kcor('z', x['z'], 'g - z', x['g_z_color']), axis=1)
galaxies['g_z_color_gkcorr'] = galaxies.apply(lambda x: calc_kcor.calc_kcor('g', x['z'], 'g - z', x['g_z_color']), axis=1)

In [147]:
galaxies[['g_z_color', 'g_z_color_gkcorr', 'g_z_color_zkcorr']].describe()

,g_z_color,g_z_color_gkcorr,g_z_color_zkcorr
count,30000.000000,30000.000000,30000.000000
mean,1.287246,0.143564,0.000954
std,0.498299,1.744422,0.313523
min,-7.216440,-67.313476,-12.851880
25%,0.953990,0.050084,-0.007559
50%,1.347330,0.118475,0.007494
75%,1.637970,0.213969,0.019160
max,7.332670,249.856235,43.987654


## Defining Sub-Sample

In [148]:
galaxies.columns

Index(['specobjid', 'plate', 'mjd', 'fiberID', 'subclass', 'z',
       'oiii_5007_flux', 'oiii_5007_flux_err', 'h_alpha_flux',
       'h_alpha_flux_err', 'h_beta_flux', 'h_beta_flux_err', 'nii_6584_flux',
       'nii_6584_flux_err', 'sii_6717_flux', 'sii_6717_flux_err',
       'sii_6731_flux', 'sii_6731_flux_err', 'oi_6300_flux',
       'oi_6300_flux_err', 'h_alpha_reqw', 'h_alpha_reqw_err',
       'neiii_3869_flux', 'neiii_3869_flux_err', 'oii_3726_flux',
       'oii_3726_flux_err', 'oii_3729_flux', 'oii_3729_flux_err', 'psfMag_g',
       'psfMagErr_g', 'psfMag_z', 'psfMagErr_z', 'nii_h_alpha',
       'nii_h_alpha_log', 'oiii_h_beta', 'oiii_h_beta_log', 'sii6717_h_alpha',
       'sii6717_h_alpha_log', 'sii6731_h_alpha', 'sii6731_h_alpha_log',
       'oi_h_alpha', 'oi_h_alpha_log', 'BPT Kauffman', 'BPT Kewley',
       'BPT final', 'sii_h_alpha', 'sii_h_alpha_log', 'h_alpha_reqw_log',
       'h_alpha_reqw_abs', 'neiii_oii', 'neiii_oii_log', 'g_z_color',
       'g_z_color_zkcorr', 'g_z_c

In [149]:
# Signal to noise ratio of neiii_3869_flux, oii_3729_flux and oii_3726_flux >= 5
# [O iii], 
# Hβ,
# [Nii], 
# Hα, 
# and[Oii] with S/N > 5, but [Ne iii]has S/N < 5

sub_gals = galaxies.loc[(galaxies['oii_3729_flux'] / galaxies['oii_3729_flux_err']) >= 5]
sub_gals = sub_gals.loc[(sub_gals['oii_3726_flux'] / sub_gals['oii_3726_flux_err']) >= 5]
sub_gals = sub_gals.loc[(sub_gals['oiii_5007_flux'] / sub_gals['oiii_5007_flux_err']) >= 5]
sub_gals = sub_gals.loc[(sub_gals['h_alpha_flux'] / sub_gals['h_alpha_flux_err']) >= 5]
sub_gals = sub_gals.loc[(sub_gals['h_beta_flux'] / sub_gals['h_beta_flux_err']) >= 5]
sub_gals = sub_gals.loc[(sub_gals['nii_6584_flux'] / sub_gals['nii_6584_flux_err']) >= 5]
sub_gals = sub_gals.loc[(sub_gals['neiii_3869_flux'] / sub_gals['neiii_3869_flux_err']) <= 5]
sub_gals

,specobjid,plate,mjd,fiberID,subclass,z,oiii_5007_flux,oiii_5007_flux_err,h_alpha_flux,h_alpha_flux_err,...,BPT final,sii_h_alpha,sii_h_alpha_log,h_alpha_reqw_log,h_alpha_reqw_abs,neiii_oii,neiii_oii_log,g_z_color,g_z_color_zkcorr,g_z_color_gkcorr
0,305174431683078144,271,51883,202,STARBURST,0.063123,92.12244,2.618963,183.4543,2.865695,...,Star Forming,0.170181,-0.769089,NaN,47.24213,0.038690,-1.412404,0.63992,-0.015457,0.014659
1,305144744869128192,271,51883,94,STARBURST,0.049476,76.97161,2.971411,1002.4060,8.275866,...,Star Forming,0.126963,-0.896324,NaN,59.72261,0.066732,-1.175666,0.95311,0.001637,0.056677
2,305131275851687936,271,51883,45,STARBURST,0.094921,56.61370,3.481388,669.6718,6.949439,...,Composite,0.145434,-0.837333,NaN,38.23195,0.113011,-0.946878,1.34757,0.005482,0.155068
3,306322322795554816,272,51941,282,STARBURST,0.043465,60.38306,3.433703,1448.7930,11.199520,...,Star Forming,0.115240,-0.938395,NaN,68.71297,0.076632,-1.115593,0.98121,0.004245,0.057867
4,305135124142385152,271,51883,59,STARFORMING,0.060961,51.07178,2.564878,143.9712,2.619117,...,Star Forming,0.162920,-0.788026,NaN,33.10934,0.069462,-1.158253,0.88379,-0.005656,0.046400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29991,814110881725048832,723,52201,310,STARFORMING,0.068147,22.09085,2.189806,243.5701,3.770745,...,Star Forming,0.173139,-0.761606,NaN,26.52466,0.171942,-0.764619,1.03748,-0.002235,0.071781
29992,814100986120398848,723,52201,274,STARFORMING,0.070792,76.34795,2.449368,132.0000,2.179709,...,Star Forming,0.178733,-0.747795,NaN,28.48208,0.066607,-1.176478,0.60287,-0.021666,0.004857
29993,814100711242491904,723,52201,273,STARBURST,0.082263,64.43201,2.950709,554.5980,5.880338,...,Star Forming,0.157734,-0.802076,NaN,39.85421,0.090721,-1.042291,0.92002,-0.015778,0.043102
29997,814066626382030848,723,52201,149,STARBURST,0.122621,29.49972,2.428270,666.6377,8.007282,...,Star Forming,0.125902,-0.899966,NaN,57.40757,0.118367,-0.926769,1.01818,-0.034960,0.044251


In [150]:
sns.displot(data=sub_gals, x='g_z_color', log_scale=False, kde=True)

plt.xlabel(r'$g-z$ Color')

 #plt.savefig('./data_histograms/distr_g_z_color.pdf')

plt.show()

In [153]:
sns.displot(data=sub_gals, x='g_z_color_gkcorr', log_scale=False, kde=True)

plt.xlabel(r'$g-z$ $k$-Corrected Color in $g$')

# plt.savefig('./data_histograms/distr_g_z_color_gkcorr.pdf')

plt.show()

In [154]:
sns.displot(data=sub_gals, x='g_z_color_zkcorr', log_scale=False, kde=True)

plt.xlabel(r'$g-z$ $k$-Corrected Color in $z$')

# plt.savefig('./data_histograms/distr_g_z_color_zkcorr.pdf')

plt.show()

## BPT Diagram

In [155]:
plot_BPT(sub_gals, 'nii_h_alpha_log', 'oiii_h_beta_log', 'BPT final', 
         r'log [NII] / H$\alpha$', r'log [OIII] / H$\beta$', 
         Kauffman=True, Kewley=True)#, save=True, save_name='BPT_sub_niiHa_oiiiHb')

## TBT Diagram

In [127]:
# drop row with g_z_color_kcorr < 10
# sub_gals.drop(sub_gals[sub_gals['g_z_color_kcorr']<-10].index, inplace=True)

In [156]:
sub_gals.columns

Index(['specobjid', 'plate', 'mjd', 'fiberID', 'subclass', 'z',
       'oiii_5007_flux', 'oiii_5007_flux_err', 'h_alpha_flux',
       'h_alpha_flux_err', 'h_beta_flux', 'h_beta_flux_err', 'nii_6584_flux',
       'nii_6584_flux_err', 'sii_6717_flux', 'sii_6717_flux_err',
       'sii_6731_flux', 'sii_6731_flux_err', 'oi_6300_flux',
       'oi_6300_flux_err', 'h_alpha_reqw', 'h_alpha_reqw_err',
       'neiii_3869_flux', 'neiii_3869_flux_err', 'oii_3726_flux',
       'oii_3726_flux_err', 'oii_3729_flux', 'oii_3729_flux_err', 'psfMag_g',
       'psfMagErr_g', 'psfMag_z', 'psfMagErr_z', 'nii_h_alpha',
       'nii_h_alpha_log', 'oiii_h_beta', 'oiii_h_beta_log', 'sii6717_h_alpha',
       'sii6717_h_alpha_log', 'sii6731_h_alpha', 'sii6731_h_alpha_log',
       'oi_h_alpha', 'oi_h_alpha_log', 'BPT Kauffman', 'BPT Kewley',
       'BPT final', 'sii_h_alpha', 'sii_h_alpha_log', 'h_alpha_reqw_log',
       'h_alpha_reqw_abs', 'neiii_oii', 'neiii_oii_log', 'g_z_color',
       'g_z_color_zkcorr', 'g_z_c

In [176]:
print(sns.color_palette('colorblind').as_hex())

['#0173b2', '#de8f05', '#029e73', '#d55e00', '#cc78bc', '#ca9161', '#fbafe4', '#949494', '#ece133', '#56b4e9']


In [220]:
# plt.figure(figsize=(12, 8))

fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, ncols=1, sharex=True,
                                    figsize=(8, 6))

ax0.scatter(sub_gals[sub_gals['BPT final']=='AGN']['neiii_oii_log'], 
            sub_gals[sub_gals['BPT final']=='AGN']['g_z_color_gkcorr'], 
            marker='s', s=22, c='#029e73', edgecolor='white', 
            label='AGN')

ax1.scatter(sub_gals[sub_gals['BPT final']=='Composite']['neiii_oii_log'], 
            sub_gals[sub_gals['BPT final']=='Composite']['g_z_color_gkcorr'], 
            marker='X', s=27, c='#de8f05', edgecolor='white', 
            label='Composite')

ax2.scatter(sub_gals[sub_gals['BPT final']=='Star Forming']['neiii_oii_log'], 
            sub_gals[sub_gals['BPT final']=='Star Forming']['g_z_color_gkcorr'], 
            marker='o', s=27, c='#0173b2', edgecolor='white', 
            label='Star Forming')

x_to_plot = np.linspace(10**(-2.7), 10**(0.5), 50)
ax0.plot(np.log10(x_to_plot), - 1.2 * np.log10(x_to_plot) - 0.4, 
         linewidth=1.2, alpha=0.8, c='k', linestyle='dashdot')
ax1.plot(np.log10(x_to_plot), - 1.2 * np.log10(x_to_plot) - 0.4, 
         linewidth=1.2, alpha=0.8, c='k', linestyle='dashdot')
ax2.plot(np.log10(x_to_plot), - 1.2 * np.log10(x_to_plot) - 0.4, 
         linewidth=1.2, alpha=0.8, c='k', linestyle='dashdot')

plt.xlabel(r'log [NeIII] / [OII]', fontsize=11)

for ax in [ax0, ax1, ax2]:
    ax.set_ylabel(r'$^0$(g-z)', fontsize=11)
    ax.set_xlim(-2, 0)
    ax.set_ylim(-0.7, 2.0)
    ax.legend()

# plt.legend(loc='upper right', fontsize=10)
# lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
# lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]

# fig.legend(lines, labels, bbox_to_anchor=(0.6, 1.02), fontsize=10)

# plt.savefig('TBT_sub_diagrams.pdf')

plt.show()